In [123]:
import pickle

import tarfile
from tqdm import tqdm_notebook as tqdm

from multiprocessing import Pool


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('russian')

from string import punctuation
# punkt= [p for p in punctuation] + ["`", "``" ,"''", "'"]
punkt= [p for p in punctuation] + ["`", "``" ,"''", "'", '"','""','"""','«','»']


import gensim
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fasttext
import pymorphy3

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split

### Базовый датасет

In [124]:
data = pd.read_csv('lenta-ru-news.csv')

data.date = pd.to_datetime(data.date)

C:\Users\bandos\AppData\Local\Temp\ipykernel_16552\1808242926.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('lenta-ru-news.csv')


In [125]:
selected_topics = ['Россия', 'Экономика', 'Силовые структуры', 'Бывший СССР', 'Спорт', 'Забота о себе', 'Строительство', 'Путешествия', 'Наука и техника']

data = data[data.topic.isin(selected_topics)]

In [126]:
data.dropna(subset=['title','topic'], inplace=True)

In [61]:
# Задаем максимальное количество новостей для каждой темы
max_news_per_topic = 10000

# Группируем данные по теме и выбираем не более 7000 новостей из каждой темы
data = data.groupby('topic').tail(max_news_per_topic)

In [127]:
data = data[['title','topic']]
data.head()

,title,topic
5,"Космонавты сомневаются в надежности ""Мира""",Россия
6,Взрыв в центре Москвы: пострадало 30 человек,Россия
7,Япония кредитует Россию на полтора миллиарда д...,Россия
9,Отмытые через Bank of NY деньги не имели отнош...,Россия
10,C 1 сентября вводится новая форма таможенной д...,Россия


In [128]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 436928 entries, 5 to 739175
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   436928 non-null  object
 1   topic   436928 non-null  object
dtypes: object(2)
memory usage: 10.0+ MB


### Добавляем пропущенные темы:

In [129]:
import pandas as pd

# Загрузите данные из titles_health_rbc_unique.csv
health_data = pd.read_csv('titles_health_rbc_unique.csv')

# Предположим, что у вас уже есть DataFrame shorted_data и health_data

# Добавьте новый столбец 'topic' в health_data с заданным значением 'Забота о себе'
health_data['topic'] = 'Забота о себе'

# Добавьте данные из health_data в shorted_data
data = pd.concat([data, health_data], ignore_index=True)


In [130]:
import pandas as pd

# Загрузите данные из titles_health_rbc_unique.csv
stroitelstvo_data = pd.read_csv('titles_Stroitelstvo_ria_unique_more_than_1000.csv')

# Предположим, что у вас уже есть DataFrame shorted_data и health_data

# Добавьте новый столбец 'topic' в stroitelstvo_data с заданным значением 'Строительство'
stroitelstvo_data['topic'] = 'Строительство'

# Добавьте данные из health_data в shorted_data
data = pd.concat([data, stroitelstvo_data], ignore_index=True)



In [131]:
data.head()

,title,topic
0,"Космонавты сомневаются в надежности ""Мира""",Россия
1,Взрыв в центре Москвы: пострадало 30 человек,Россия
2,Япония кредитует Россию на полтора миллиарда д...,Россия
3,Отмытые через Bank of NY деньги не имели отнош...,Россия
4,C 1 сентября вводится новая форма таможенной д...,Россия


In [132]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440165 entries, 0 to 440164
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   440165 non-null  object
 1   topic   440165 non-null  object
dtypes: object(2)
memory usage: 6.7+ MB


In [133]:
topic_counts = data['topic'].value_counts()
topic_percentage = data['topic'].value_counts(normalize=True) * 100

result_df = pd.DataFrame({
    'Topic': topic_counts.index,
    'Count': topic_counts.values,
    'Percentage': topic_percentage.values
})

print(result_df)

               Topic   Count  Percentage
0             Россия  160445   36.451104
1          Экономика   79528   18.067770
2              Спорт   64413   14.633830
3        Бывший СССР   53402   12.132269
4    Наука и техника   53136   12.071837
5  Силовые структуры   19596    4.451967
6        Путешествия    6408    1.455818
7      Строительство    2344    0.532528
8      Забота о себе     893    0.202878


In [134]:
# # Задайте путь к файлу CSV, в который вы хотите записать объединенные данные
csv_filename_combined = "news_data_440k.csv"

# # Записываем объединенные данные в файл CSV
data.to_csv(csv_filename_combined, index=False)

print(f"Combined data has been successfully written to {csv_filename_combined}.")

Combined data has been successfully written to news_data_440k.csv.


### Теперь можно начинать отсюда 

In [135]:
data = pd.read_csv("news_data_440k.csv")

In [136]:

# data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440165 entries, 0 to 440164
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   440165 non-null  object
 1   topic   440165 non-null  object
dtypes: object(2)
memory usage: 6.7+ MB


In [137]:
topic_counts = data['topic'].value_counts()
topic_percentage = data['topic'].value_counts(normalize=True) * 100

result_df = pd.DataFrame({
    'Topic': topic_counts.index,
    'Count': topic_counts.values,
    'Percentage': topic_percentage.values
})

print(result_df)

               Topic   Count  Percentage
0             Россия  160445   36.451104
1          Экономика   79528   18.067770
2              Спорт   64413   14.633830
3        Бывший СССР   53402   12.132269
4    Наука и техника   53136   12.071837
5  Силовые структуры   19596    4.451967
6        Путешествия    6408    1.455818
7      Строительство    2344    0.532528
8      Забота о себе     893    0.202878


### Начало обработки данных

In [138]:
topic_mapping = {
    'Россия': 0,
    'Экономика': 1,
    'Силовые структуры': 2,
    'Бывший СССР': 3,
    'Спорт': 4,
    'Забота о себе': 5,
    'Строительство': 6,
    'Путешествия': 7,
    'Наука и техника': 8
}

# Заменить значения в колонке 'topic' на их числовые представления из словаря
data['topic'] = data['topic'].replace(topic_mapping)

In [139]:
topic_counts = data['topic'].value_counts()
topic_percentage = data['topic'].value_counts(normalize=True) * 100

result_df = pd.DataFrame({
    'Topic': topic_counts.index,
    'Count': topic_counts.values,
    'Percentage': topic_percentage.values
})

print(result_df)

   Topic   Count  Percentage
0      0  160445   36.451104
1      1   79528   18.067770
2      4   64413   14.633830
3      3   53402   12.132269
4      8   53136   12.071837
5      2   19596    4.451967
6      7    6408    1.455818
7      6    2344    0.532528
8      5     893    0.202878


### Начало обучения наших моделей

1 способ

In [151]:
titles = data['title']
topics = data['topic']
X = titles.tolist()
Y = topics.tolist()
len(X),len(Y)

(440165, 440165)

In [141]:
with open('glob_train_data_titles.txt', 'w', encoding='utf-8') as tr:
    for idx in range(len(X)):
        # tr.write('__label__' + y_train[idx]+' '+title_train[idx]+'\n')
        tr.write('__label__' + f'{Y[idx]}'+' '+X[idx]+'\n')

In [142]:
%%time
ft_model_p1 = fasttext.train_supervised(input='glob_train_data_titles.txt', epoch=20, wordNgrams=2, loss='hs', lr=1.0)

CPU times: total: 3min 2s
Wall time: 36 s


In [149]:
test_data = pd.read_csv('lemm_test_news.csv')

In [144]:
# Предсказание меток для каждого заголовка новости
predicted_labels = []

for title in test_data['content']:  # Замените 'content' на реальное имя столбца с заголовками
    # Удаление символа новой строки, если он присутствует
    title = title.replace('\n', ' ')
    # Предсказание меток
    # labels, probs = ft_model_p1.predict(title)
    labels, probs = ft_model_p1.predict(title)
    # print(labels, probs)
    predicted_labels.append(labels[0])

# Добавление предсказанных меток в тестовые данные
test_data['predicted_label'] = predicted_labels

In [145]:
test_data.head()

,content,predicted_label
0,фото фонтанка.р поделитьсяэкс-министр оборона ...,__label__0
1,начало февраль 2023 год пушкинский район санк...,__label__1
2,фото andy bao getty images анастасия борисов ...,__label__4
3,хотеть съездить море летом-2023 — читать далё...,__label__0
4,сергей пиняев фото алексей филипп риа новость...,__label__4


In [147]:
test_data['topic'] = test_data['predicted_label'].copy().str.replace('__label__', '').astype(int)
test_data['index'] = test_data.index.copy() 
test_data = test_data[['topic','index']].copy()

test_data.head()


,topic,index
0,0,0
1,1,1
2,4,2
3,0,3
4,4,4


In [148]:
test_data.to_csv('result_6.csv', index=False)

### Проверим точность на своих тренировочных данных: (20% от общих)

In [162]:
title_train, title_test, y_train, y_test = train_test_split(X, Y, test_size=0.05, stratify=Y, random_state=42)

In [161]:
with open('train_data_titles.txt', 'w', encoding='utf-8') as tr:
    for idx in range(len(title_train)):
        # tr.write('__label__' + y_train[idx]+' '+title_train[idx]+'\n')
        tr.write('__label__' + f'{y_train[idx]}'+' '+title_train[idx]+'\n')
        
with open('test_data_titles.txt', 'w', encoding='utf-8') as te:
    for idx in range(len(title_test)):
        # te.write('__label__' + y_test[idx]+' '+title_test[idx]+'\n')        
        te.write('__label__' + f'{y_test[idx]}'+' '+title_test[idx]+'\n')    
print('Все записано!')

Все записано!


In [91]:
%%time
# ft_model_p1 = fasttext.train_supervised(input='train_data_titles.txt', epoch=20, wordNgrams=2, loss='hs', lr=1.0)
ft_model_p1 = fasttext.train_supervised(input='train_data_titles.txt', epoch=20, wordNgrams=2, loss='hs', lr=1.0)

CPU times: total: 29.7 s
Wall time: 6.29 s


In [92]:
ft_model_p1.test('test_data_titles.txt')

# 0.92 пока лучшее (epoch=20, wordNgrams=2, loss='hs', lr=1.0)

(13929, 0.9243305334194846, 0.9243305334194846)

### Более лучшая модель

In [157]:
%%time
ft_model_p2 = fasttext.train_supervised(input='train_data_titles.txt', autotuneValidationFile='test_data_titles.txt')

CPU times: total: 22min 58s
Wall time: 5min 11s


In [158]:
ft_model_p2.test('test_data_titles.txt')

(88033, 0.8737064509899697, 0.8737064509899697)

In [159]:
print(f'Best learning rate: {ft_model_p2.lr} \nBest epochs: {ft_model_p2.epoch} \nBest_word_Ngrams: {ft_model_p2.wordNgrams}')

Best learning rate: 1.2012255826641023 
Best epochs: 3 
Best_word_Ngrams: 3


In [163]:
# test_data = pd.read_csv('lemm_test_news.csv')  # порпобуем на грязненьком
test_data = pd.read_csv('test_news.csv')
# Предсказание меток для каждого заголовка новости
predicted_labels = []

for title in test_data['content']:  # Замените 'content' на реальное имя столбца с заголовками
    # Удаление символа новой строки, если он присутствует
    title = title.replace('\n', ' ')
    # Предсказание меток
    labels, probs = ft_model_p2.predict(title)
    # print(labels, probs)
    predicted_labels.append(labels[0])

# Добавление предсказанных меток в тестовые данные
test_data['predicted_label'] = predicted_labels
test_data.head()



,content,predicted_label
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...,__label__6
1,В начале февраля 2023 года в Пушкинском районе...,__label__6
2,Фото: Andy Bao / Getty Images Анастасия Борисо...,__label__5
3,"Если вы хотели, но так и не съездили на море л...",__label__5
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...,__label__4


In [164]:
test_data['topic'] = test_data['predicted_label'].copy().str.replace('__label__', '').astype(int)
test_data['index'] = test_data.index.copy() 
test_data = test_data[['topic','index']].copy()

test_data.head()

,topic,index
0,6,0
1,6,1
2,5,2
3,5,3
4,4,4


In [165]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26275 entries, 0 to 26274
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   topic   26275 non-null  int32
 1   index   26275 non-null  int64
dtypes: int32(1), int64(1)
memory usage: 308.0 KB


In [166]:
test_data.to_csv('result_7.csv', index=False)